In [1]:
import numpy as np
import scipy as sp
import sympy as sy
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy import signal
import math
import statistics
from numpy import trapz

In [20]:
def Model_Single(name,hl_lumen1,hl_lumen2,hl_lumen3,Cl_int_invitro_liver,Cl_int_invitro_intestine,logKow,MW,Clint_type,Papp_caco2):
    
    k_transit = 3/4 # 4hr small intestine transit time, Basic Anatomical and Physiological Data for Use in Radiological Protection: Reference Values, pg128
   
    Peff = 1.07595 * 1.75 * 1/2 * 1/4 * 1/3600 * Papp_caco2 ** 0.29395 * (1e6) ** 0.29395

    Klipidw = 10**(logKow)
    Kproteinw = 10**(0.73*logKow-0.39)
    Kplipidw = 10**(1.01*logKow+0.12)
    Kalbuminw = 10**(0.73*logKow-0.39)

# Kliver2plasma
#liver comp
    Flw = 0.73
    Flnl = 0.019
    Flpl = 0.046
    Fla = 0.0019
    Flp = 0.17
#plasma comp
    Fpw = 0.96
    Fpnl = 0.0015
    Fppl = 0.0008
    Fpa = 0.029
    Fpp = 0.015

    Kliverwater = Flw + Flnl*Klipidw + Flpl*Kplipidw + Fla*Kalbuminw + Flp*Kproteinw
    
    Kplasmawater = Fpw + Fpnl*Klipidw + Fppl*Kplipidw + Fpa*Kalbuminw + Fpp*Kproteinw
    
    K_liver2plasma = Kliverwater/Kplasmawater

#Kgut2plasma    
#gut comp
    Fgw = 0.7969
    Fgnl = 0.04347
    Fgpl = 0.01953
    Fga = 0.001056
    Fgp = 0.135


    Kgutwater = Fgw + Fgnl*Klipidw + Fgpl*Kplipidw + Fga*Kalbuminw + Fgp*Kproteinw
    K_gut2plasma = Kgutwater/Kplasmawater

    

# Krest2plasma
#     Kmuscle2plasma + Klung2plasma + Kbrain2plasma + Kkidney2plasma + Kblood2plasma
# Kmuscle2plasma
    Fmw = 0.788
    Fmnl = 0.0043
    Fmpl = 0.0045
    Fma = 0.0013
    Fmp = 0.17
    Kmusclewater = Fmw + Fmnl*Klipidw + Fmpl*Kplipidw + Fma*Kalbuminw + Fmp*Kproteinw
    K_muscle2plasma = Kmusclewater/Kplasmawater
    
#Klung2plasma

    Fluw = 0.84
    Flunl = 0.0102
    Flupl = 0.0098
    Flua = 0.0054
    Flup = 0.055
    Klungwater = Fluw + Flunl*Klipidw + Flupl*Kplipidw + Flua*Kalbuminw + Flup*Kproteinw
    K_lung2plasma = Klungwater/Kplasmawater

#Kbrain2plasma
    Fbw = 0.79
    Fbnl = 0.043
    Fbpl = 0.067
    Fba = 0.00004
    Fbp = 0.08
    Kbrainwater = Fbw + Fbnl*Klipidw + Fbpl*Kplipidw + Fba*Kalbuminw + Fbp*Kproteinw
    K_brain2plasma = Kbrainwater/Kplasmawater
#KKidney2plasma
    Fkw = 0.78
    Fknl = 0.012
    Fkpl = 0.035
    Fka = 0.0024
    Fkp = 0.16
    Kkidneywater = Fkw + Fknl*Klipidw + Fkpl*Kplipidw + Fka*Kalbuminw + Fkp*Kproteinw
    K_kidney2plasma = Kkidneywater/Kkidneywater
#Kblood2plasma   
    Fblw = 0.81
    Fblnl = 0.0013
    Fblpl = 0.0022
    Fbla = 0.016
    Fblp = 0.16
    Kbloodwater = Fblw + Fblnl*Klipidw + Fblpl*Kplipidw + Fbla*Kalbuminw + Fblp*Kproteinw
    K_blood2plasma = Kbloodwater/Kplasmawater
    K_rest2plasma = (0.3842*K_muscle2plasma + 0.007235*K_lung2plasma + 0.019310*K_brain2plasma + 0.00419*K_kidney2plasma)/(0.3842+0.007235+0.019310+0.00419)
    
#fraction unbound 2 plasma    
    fup = (1/Kplasmawater)*Fpw
    fub_blood = (1/Kbloodwater)*Fblw
  
    R_blood2plasma = K_blood2plasma

    Vr = 0.5
    BW = 70
    Q_cardiacc = 13.88 * BW**0.75 # L/h/kg BW
    Q_gut = 0.2054 * Q_cardiacc  # Fraction of cardiac output flowing to the gut
#     Q_gut = 0.000000000001 * Q_gut
    Q_rest = 0.7221 * Q_cardiacc
    V_intestine = 8.9e-3 * BW #Volume of the gut per kg body weight, L/kg BW Basic Anatomical and Physiological Data for Use in Radiological Protection: Reference Values table 6.6, small intestine mass/ body mass, male
    Q_liver = 0.05359*Q_cardiacc # Fraction of cardiac output flowing to the liver
    V_liver = 0.0245* BW  #Volume of the liver per kg body weight, L/kg BW.
    V_rest = 0.788 * BW#Volume of the rest of the body per kg body weight, L/kg BW.
    Q_gfr = 0.30998 * (BW ** (3/4))#* Q_cardiacc #  (BW/(BW**0.75)) #Glomerular Filtration Rate, L/day/kg BW^3/4
    
    F_reabsorption = (1 - 0.06 / Q_gfr) * Papp_caco2 ** 2.7 / (34.4E-6 ** 2.7+ Papp_caco2 ** 2.7) # https://doi.org/10.1016/j.ejps.2016.03.018
    
    Q_gfr = Q_gfr * (1-F_reabsorption) # Dec 1

    V_lumen = 2
    V_content = 0.26  #Basic Anatomical and Physiological Data for Use in Radiological Protection: Reference Values pg131
    Q_transit = V_content/4
    
    R = 1.75 # cm
    
    k_gutabs = (2 * Peff/ R) * 3600 
        
    Vr = 0.05
    
    #adjust for fraction unbound in assay , microsomes in this case based on Alessandro Sangion <
    
    Fassaya = 0.0
    Fassaynl = 0
    Fassayp = (1/1000)
    Fassaypl = 0.35*Fassayp
    Fassayw = 1-Fassayp-Fassaypl-Fassaya-Fassaynl
    
    Kassayw = Fassayw + Fassaynl*Klipidw + Fassaypl*Kplipidw + Fassaya*Kalbuminw + Fassayp*Kproteinw
    
    fu_mic = (1/Kassayw)*Fassayw

    Cp = 1 # protein concentration in essay, default to 1mg/ml\

    fub = fup
#    
    wwater = 1 
    fub_hep = 1/(1+ 125*Vr*10**(0.072*logKow**2+0.067*logKow-1.126)) #Drug Metabolism and Disposition July 2008, 36 (7) 1194-1197; DOI: https://doi.org/10.1124/dmd.108.020834

    
    if Clint_type == "microsome":
        Cl_protein = Cl_int_invitro_liver *62.9*24.9*BW*60/(1e6) # doi:10.3109/03639045.2013.831439      table 4
        Cl_invivo_liver = Cl_protein * wwater/fu_mic  #IN unit of L/h for a 70kg human # unscaled
        HL_invivo_liver = np.log(2)/(Cl_invivo_liver/V_liver)
    
    elif Clint_type == "hep":
        Cl_protein = Cl_int_invitro_liver* BW * 110*1000 * 0.0245 * 1.05 *60 / (10**6) # hepatocyte
        Cl_invivo_liver = Cl_protein * wwater/fub_hep  #IN unit of L/h for a 70kg human # unscaled
        HL_invivo_liver = np.log(2)/(Cl_invivo_liver/V_liver)

    else: print ("error")

        
    Cl_invivo_intestine = wwater * Cl_int_invitro_intestine * 3.9 * (650/73)*BW * 1e-6 * 60/fu_mic # [Cl_int_invitro_intestine]=uL/mgprotein/min, 3155 mgprotein/intestine (hatley 2017), Basic Anatomical and Physiological Data for Use in Radiological Protection 650g 
    HL_invivo_intestine = np.log(2)/(Cl_invivo_intestine/V_intestine)    
    #"Different from Goss and others because of division by unbound fractions
    
    
    HL_invivo_lumen = hl_lumen3
    Cl_invivo_lumen = (np.log(2)/HL_invivo_lumen)*V_content

    # Cl_organ, only not consider blood flow

    Cl_liver = Cl_invivo_liver * fub_blood  #IN unit of L/h for a 70kg human # unscaled
    HL_liver = np.log(2)/(Cl_liver/V_liver)


        
    Cl_intestine = Cl_invivo_intestine * fub_blood
    HL_intestine = np.log(2)/(Cl_intestine/V_intestine)    
    #"Different from Goss and others because of division by unbound fractions
    
    
    HL_lumen = hl_lumen3
    Cl_lumen = (np.log(2)/HL_invivo_lumen)*V_content

#  blood flow limited 
#   hepatic clearence L/hr
    Cl_liver_flow = ((((Q_liver+Q_gut)*Cl_liver)/((Q_liver+Q_gut) + Cl_liver))) # blood flow limited

#     Cl_liver_flow = (((Q_liver)*Cl_liver*K_liver2plasma/((Q_liver) + Cl_liver*K_liver2plasma))) # blood flow limited
    HL_liver_flow = np.log(2)/(Cl_liver_flow/V_liver) # blood flow limited
    
#   intestinal clearence L/hr
    Cl_intestine_flow = ((Q_gut*Cl_intestine)/(Q_gut + Cl_intestine)) #blood flow limited
    HL_intestine_flow = np.log(2)/(Cl_intestine_flow/V_intestine)
    
#   lumen clearence
    
    Cl_lumen_flow = Cl_invivo_lumen*Q_transit/(Cl_invivo_lumen + Q_transit)
    HL_lumen_flow = np.log(2)/(Cl_lumen_flow/V_content)

    fuG = Fgw/(Kgutwater)
    ClintG = Cl_invivo_intestine
    Clperm = 10**(0.4926*np.log10(Papp_caco2)-0.1457)*(0.1*3600)*(0.66*10) #[L/hr]
    Clperm = Papp_caco2*(0.1*3600)*(0.66*10) #[L/hr]

    Qvilli = 298 #mL/min
    Qvilli = Qvilli*(60/1000)# [L/hr] 
    QGUT = Q_gut
    FG = QGUT/(QGUT+fuG * ClintG)
    
    QGUT = Qvilli * Clperm/(Qvilli + Clperm)
    FG = QGUT/(QGUT+fuG * ClintG)
    
    def model(z,t):
    
        dA_lumen1 = -k_gutabs * z[0] -k_transit * z[0] - (np.log(2)/hl_lumen1) * z[0] 
        dA_lumen2 = +k_transit * z[0] -k_transit * z[1] - k_gutabs * z[1] - (np.log(2)/hl_lumen2) * z[1]  
        dA_lumen3 = +k_transit * z[1] -k_transit * z[2] - k_gutabs * z[2] - (np.log(2)/hl_lumen3) * z[2]  

        dC_gut = +(k_gutabs * (z[0] + z[1]+ z[2]) + ((Q_gut * R_blood2plasma)/(K_rest2plasma)) * z[5]
                  - ((Q_gut * R_blood2plasma)/(K_gut2plasma) +  Cl_intestine*R_blood2plasma/K_gut2plasma)* z[3])/ (V_intestine)

        dC_liver = + (((Q_gut * R_blood2plasma)/(K_gut2plasma)) * z[3] 
                    + ((Q_liver * R_blood2plasma)/(K_rest2plasma)) * z[5]
                    - ((((Q_liver + Q_gut)*R_blood2plasma)/(K_liver2plasma)) + Cl_liver*R_blood2plasma/K_liver2plasma) * z[4])/ (V_liver)

        dC_rest = (((Q_gut + Q_liver)*R_blood2plasma/(K_liver2plasma)) * z[4] - ((R_blood2plasma)*(Q_gut + Q_liver) 
                    + (Q_gfr * fup)) * z[5]/K_rest2plasma) / (V_rest)
        
                    
        dzdt = [dA_lumen1,dA_lumen2,dA_lumen3,dC_gut,dC_liver,dC_rest]
        
        return dzdt

#
    z0 = [1,0,0,0,0,0] 


    # time point
    t = np.linspace(0,24,10000)

    # solve ODE
    z = odeint(model,z0,t)
 